# Corporate Actions in LUSID

LUSID will automatically update portfolios that are affected by Corporate Actions upserted into the system. More information is detailed in the support page below.

[LUSID Knowledge Base: Corporate Actions](https://support.lusid.com/how-are-corporate-actions-represented)

This example will show how to :
1. Create Corporate Action Sources
2. Load required Data from an example CSV
3. Create and Upsert portfolios and relevant transactions
4. Upsert Corporate Actions
5. Verify Corporate Actions have been correctly applied

Initialise our environment and connect to LUSID

*Run the cell below to initialise the environment*

In [1]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import uuid
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os
from msrest.authentication import BasicTokenAuthentication

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print('LUSID Environment Initialised')
print('LUSID version : ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID version :  0.5.3239.0


_Run the cell below to set up some useful functions and dates used in this notebook_

In [2]:
# The following function creates a random alphanumeric code of 4 characters that can be appended to Ids
# and Names to ensure they remain unique throughout multiple runs of this example
def GetGuid():
    return str(uuid.uuid4())[:4]

#some useful dates
Mar01 = datetime(2018, 3, 1, tzinfo=pytz.utc)
Mar05 = datetime(2018, 3, 5, tzinfo=pytz.utc)
Mar08 = datetime(2018, 3, 8, tzinfo=pytz.utc)
Mar09 = datetime(2018, 3, 9, tzinfo=pytz.utc)
Mar10 = datetime(2018, 3, 10, tzinfo=pytz.utc)
Mar11 = datetime(2018, 3, 11, tzinfo=pytz.utc)
Mar13 = datetime(2018, 3, 13, tzinfo=pytz.utc)
Mar15 = datetime(2018, 3, 15, tzinfo=pytz.utc)
Mar16 = datetime(2018, 3, 16, tzinfo=pytz.utc)
Mar18 = datetime(2018, 3, 18, tzinfo=pytz.utc)

## 1. Corporate Action Sources

Corporate Actions in LUSID are upserted to a specific Corporate Action Source, which can be thought of as a container for a set of actions, which portfolios can subscribe to for automatic updates. Sources should be created first, though Portfolios can also be created without a source and this detail can be added at a later time. 

First we will create two scopes, and two codes with randomly generated 4 character suffixes that will be used for CASources and Portfolios throughout this notebook.

_Run the cell below to do this_

In [3]:
# Create some demo scopes
scope1 = 'ca_demo_abc'
scope2 = 'ca_demo_xyz'
print("Scope1 : ", scope1)
print("Scope2 : ", scope2)

# Create corporate action sources
# First create a couple of codes that match the scopes, make these unique by adding a random guid onto the end
guid = GetGuid()
code1 = scope1+'_'+guid
code2 = scope2+'_'+guid

print('')
print("Code1 : ", code1)
print("Code2 : ", code2)

Scope1 :  ca_demo_abc
Scope2 :  ca_demo_xyz

Code1 :  ca_demo_abc_b7ba
Code2 :  ca_demo_xyz_b7ba


_Run the cell below to create two Sources, one for each scope_

In [4]:
# Create first corporate action source 
source1_request = models.CreateCorporateActionSourceRequest(
    scope=scope1,                                                             
    code=code1, 
    display_name="PNB_CASource1", 
    description="CASource 1 for python notebook demo module")
source1_result = client.corporate_action_sources.create_corporate_action_source(source1_request)
CASource1 = source1_result.id
print('Corporate Action Source Created : ')
print('Disply Name: ', source1_result.display_name)
print('Description: ', source1_result.description)
print('Scope: ', CASource1.scope)
print('Code: ', CASource1.code + '\n')

# Create second corporate action source
source2_request = models.CreateCorporateActionSourceRequest(
    scope=scope2,                                                             
    code=code2, 
    display_name="PNB_CASource2", 
    description="CASource 2 for python notebook demo module")
source2_result = client.corporate_action_sources.create_corporate_action_source(source2_request)
CASource2 = source2_result.id
print('Corporate Action Source Created : ')
print('Disply Name: ', source2_result.display_name)
print('Description: ', source2_result.description)
print('Scope: ', CASource2.scope)
print('Code: ', CASource2.code)

Corporate Action Source Created : 
Disply Name:  PNB_CASource1
Description:  CASource 1 for python notebook demo module
Scope:  ca_demo_abc
Code:  ca_demo_abc_b7ba

Corporate Action Source Created : 
Disply Name:  PNB_CASource2
Description:  CASource 2 for python notebook demo module
Scope:  ca_demo_xyz
Code:  ca_demo_xyz_b7ba


It is also possible to list all existing sources.

_Run the cell below to list all sources_

In [5]:
all_sources = client.corporate_action_sources.list_corporate_action_sources(sort_by="scope")

print('Number of sources in client: ' + str(len(all_sources.values)))
for s in all_sources.values:
     print("Scope : " + s.id.scope)
     print("Code : " + s.id.code + '\n')


Number of sources in client: 4
Scope : ca_demo_xyz
Code : ca_demo_xyz_b7ba

Scope : ca_demo_abc
Code : ca_demo_abc_b7ba

Scope : ExampleScope
Code : DataLoader

Scope : ExampleScope
Code : ExampleCode



## 2. Loading Data from CSV
In order to demonstrate how Corporate Actions are applied in LUSID, we need to load some instruments that will be affected by these Corporate Actions first.

_Run the cells below to load and upsert instruments to LUSID_

In [6]:
# Read instruments from a CSV to a Pandas dataframe
instruments = pd.read_csv('data/corporateactions-instruments.csv')

_Run the cell below to print the instruments dataframe_

In [7]:
instruments.head(10)

instrument_name client_internal currency          isin  \
0          Amazon_Nasdaq_AMZN    imd_34634534      USD  US0231351067   
1           Apple_Nasdaq_AAPL    imd_35345345      USD  US0378331005   
2    ExpressScripts_NYSE_ESRX    imd_34352311      USD  US30219G1085   
3  TrinityIndustries_NYSE_TRN    imd_34235200      USD  US8965221091   
4              Trex_NYSE_TREX    imd_32423956      USD  US89531P1057   
5               Cigna_NYSE_CI    imd_32452391      USD  US1255091092   
6             Arcosa_NYSE_ACA    imd_23423409      USD  US0396531008   

           figi exchange_code          country_issue ticker market_sector  \
0  BBG000BVPXP1            UN  united_states_america   AMZN        equity   
1  BBG000B9XVV8            UN  united_states_america   AAPL        equity   
2  BBG000C16621            UN  united_states_america   ESRX        equity   
3  BBG000BVL406            UN  united_states_america    TRN        equity   
4  BBG000BTGM43            UN  united_states_america   TREX        equity   
5  BBG00KXXK940            UN  united_states_america     CI        equity   
6  BBG00JGMWFQ5            UN  united_states_america    ACA        equity   

  security_type  coupon  
0  common_stock     NaN  
1  common_stock     NaN  
2  common_stock     NaN  
3  common_stock     NaN  
4  common_stock     NaN  
5  common_stock     NaN  
6  common_stock     NaN

In [8]:
definitions = {}

# Specify your identifier columns
identifier_columns = [
    ('isin', 'Isin'),
    ('figi', 'Figi'),
    ('ticker', 'Ticker'),
    ('client_internal', 'ClientInternal')
]

# step through the data row by row to create instrument properties and definitions
for row, instrument in instruments.iterrows():
    print("Creating instrument : ", instrument['instrument_name'])
    
    # Create Identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier[1]] = models.InstrumentIdValue(
            value=instrument[identifier[0]])
        
    # Create definitions
    definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers,
        properties=None,
        look_through_portfolio_id=None)

# Call the LUSID API to upsert our instrument definitions
response = client.instruments.upsert_instruments(requests=definitions)
# Check that there aren't any failures, if there are, print them out
for f in response.failed:
    print(response.failed[f])

# Print the definition of each instrument returned in the response
luids = []
print('')
for name, instrument in response.values.items():
    print('Figi: ' + instrument.identifiers['Figi'] + '\n',
          'Instrument Name: ' + name + '\n',
          'LUSID Instrument ID: ' + instrument.lusid_instrument_id + '\n'+'\n')
    luids.append((instrument.lusid_instrument_id, instrument.identifiers['Figi']))

Creating instrument :  Amazon_Nasdaq_AMZN
Creating instrument :  Apple_Nasdaq_AAPL
Creating instrument :  ExpressScripts_NYSE_ESRX
Creating instrument :  TrinityIndustries_NYSE_TRN
Creating instrument :  Trex_NYSE_TREX
Creating instrument :  Cigna_NYSE_CI
Creating instrument :  Arcosa_NYSE_ACA

Figi: BBG000BVL406
 Instrument Name: TrinityIndustries_NYSE_TRN
 LUSID Instrument ID: LUID_GH8D5K6V


Figi: BBG000C16621
 Instrument Name: ExpressScripts_NYSE_ESRX
 LUSID Instrument ID: LUID_S6ANN8K2


Figi: BBG000B9XVV8
 Instrument Name: Apple_Nasdaq_AAPL
 LUSID Instrument ID: LUID_KVBLERMO


Figi: BBG00KXXK940
 Instrument Name: Cigna_NYSE_CI
 LUSID Instrument ID: LUID_WCV2P2VV


Figi: BBG000BVPXP1
 Instrument Name: Amazon_Nasdaq_AMZN
 LUSID Instrument ID: LUID_8E4NW6LN


Figi: BBG00JGMWFQ5
 Instrument Name: Arcosa_NYSE_ACA
 LUSID Instrument ID: LUID_E8ZMJAH7


Figi: BBG000BTGM43
 Instrument Name: Trex_NYSE_TREX
 LUSID Instrument ID: LUID_CATB74OJ




_Run the cell below to load the corporate actions information csv files and print the dataframe_

In [9]:
#load corporate action info
ca_info = pd.read_csv('data/corporateactions.csv')
ca_info.head(10)


code action_description                             description  \
0  5943592342           dividend                          Apple Dividend   
1  5943592343             merger          Cigna & Express Scripts Merger   
2  5943592343             merger          Cigna & Express Scripts Merger   
3  5943592347           spin-off  Acosa spin-off from Trinity Industries   
4  5943592498              split                Trex 2 for 1 stock split   

  announcement_date     ex_date record_date payment_date  \
0        29/01/2019  02/08/2019  02/11/2019   14/02/2019   
1        03/08/2018  12/10/2018  13/12/2018   20/12/2018   
2        03/08/2018  12/10/2018  13/12/2018   20/12/2018   
3        12/12/2017  17/10/2018  20/10/2018   11/01/2018   
4        05/05/2018  21/05/2018  23/05/2018   19/06/2018   

  input_instrument_figi instrument_input_ticker        input_instrument_name  \
0          BBG000B9XVV8                    AAPL                    APPLE INC   
1          BBG000C16621                    ESRX  EXPRESS SCRIPTS HOLDINGS CO   
2          BBG000C16621                    ESRX  EXPRESS SCRIPTS HOLDINGS CO   
3          BBG000BVL406                     TRN       TRINITY INDUSTRIES INC   
4          BBG000BTGM43                    TREX             TREX COMPANY INC   

   input_units_factor  input_cost_factor output_instrument_figi  \
0                   1                  1                    NaN   
1                   1                  1                    NaN   
2                   1                  1           BBG00KXXK940   
3                   3                  1           BBG00JGMWFQ5   
4                   1                  1           BBG000BTGM43   

  output_instrument_name output_ticker output_instrument_internal  \
0                    NaN           NaN                    CCY_GBP   
1                    NaN           NaN                    CCY_USD   
2             CIGNA CORP            CI                        NaN   
3             ARCOSA INC           ACA                        NaN   
4       TREX COMPANY INC          TREX                        NaN   

   output_units_factor  output_cost_factor  dividend_yield  
0               0.0170              1.0000           0.017  
1              48.7500              1.0000             NaN  
2               0.2434              1.9337             NaN  
3               1.0000              1.0324             NaN  
4               2.0000              1.0000             NaN

As you can see from the cell above, we are going to create four Corporate Actions. The original CSV contains the input/output instrument figis, which can be used on their own to upsert Corporate Actions. Alternatively, ISIN, RIC, CUSIP and internal Ids can also be used, provided they exist within LUSID, though since some of these may be non-unique, it is always best to use LUIDs instead. In this example we will use both FIGI and LUID for demonstration purposes.

_Run the cell below to create another dataframe of CA info and replace the FIGIs with LUIDs_

In [10]:
# replace figis with luids for simplicity further on
luid_cas = ca_info
for row in luids:
    luid_cas.replace(to_replace=row[1], value=row[0], inplace=True)

luid_cas = luid_cas.rename(index=str, 
              columns={
                  "input_instrument_figi": "input_instrument_luid", 
                  "output_instrument_figi": "output_instrument_luid"})
luid_cas.head()

code action_description                             description  \
0  5943592342           dividend                          Apple Dividend   
1  5943592343             merger          Cigna & Express Scripts Merger   
2  5943592343             merger          Cigna & Express Scripts Merger   
3  5943592347           spin-off  Acosa spin-off from Trinity Industries   
4  5943592498              split                Trex 2 for 1 stock split   

  announcement_date     ex_date record_date payment_date  \
0        29/01/2019  02/08/2019  02/11/2019   14/02/2019   
1        03/08/2018  12/10/2018  13/12/2018   20/12/2018   
2        03/08/2018  12/10/2018  13/12/2018   20/12/2018   
3        12/12/2017  17/10/2018  20/10/2018   11/01/2018   
4        05/05/2018  21/05/2018  23/05/2018   19/06/2018   

  input_instrument_luid instrument_input_ticker        input_instrument_name  \
0         LUID_KVBLERMO                    AAPL                    APPLE INC   
1         LUID_S6ANN8K2                    ESRX  EXPRESS SCRIPTS HOLDINGS CO   
2         LUID_S6ANN8K2                    ESRX  EXPRESS SCRIPTS HOLDINGS CO   
3         LUID_GH8D5K6V                     TRN       TRINITY INDUSTRIES INC   
4         LUID_CATB74OJ                    TREX             TREX COMPANY INC   

   input_units_factor  input_cost_factor output_instrument_luid  \
0                   1                  1                    NaN   
1                   1                  1                    NaN   
2                   1                  1          LUID_WCV2P2VV   
3                   3                  1          LUID_E8ZMJAH7   
4                   1                  1          LUID_CATB74OJ   

  output_instrument_name output_ticker output_instrument_internal  \
0                    NaN           NaN                    CCY_GBP   
1                    NaN           NaN                    CCY_USD   
2             CIGNA CORP            CI                        NaN   
3             ARCOSA INC           ACA                        NaN   
4       TREX COMPANY INC          TREX                        NaN   

   output_units_factor  output_cost_factor  dividend_yield  
0               0.0170              1.0000           0.017  
1              48.7500              1.0000             NaN  
2               0.2434              1.9337             NaN  
3               1.0000              1.0324             NaN  
4               2.0000              1.0000             NaN

## 3. Portfolios and Transactions
#### 3.1 Creating Portfolios
The next step is to create some portfolios and populate them with the instruments affected by the Corporate Actions above. To demonstrate some of the functionality of Corporate Actions and Corporate Action Sources, we will create one portfolio in each scope created earlier, applying CASource 1 to one of the portfolios, and leaving the second portfolio without a Corporate Action Source. 

_Run the cell below to create these two portfolios_

In [11]:
# Create a portfolio with a corporate action source on it
effective_date = datetime(2018, 1, 1, tzinfo=pytz.utc)
p1_request = models.CreateTransactionPortfolioRequest(
            code='portfolio-'+GetGuid(),
            display_name='portfolio-A',
            base_currency='GBP',
            created=effective_date,
            description=None,
            corporate_action_source_id=CASource1,
            accounting_method=None,
            sub_holding_keys=None,
            properties=None)
p1_result = client.transaction_portfolios.create_portfolio(
            scope=scope1,
            create_request=p1_request)

# Create a portfolio without a corporate action source
p2_request = models.CreateTransactionPortfolioRequest(
            code='portfolio-'+GetGuid(),
            display_name='portfolio-B',
            base_currency='GBP',
            created=effective_date,
            description=None,
            corporate_action_source_id=None,
            accounting_method=None,
            sub_holding_keys=None,
            properties=None)
p2_result = client.transaction_portfolios.create_portfolio(
            scope=scope2,
            create_request=p2_request)

prettyprint.portfolio_response(p1_result)
prettyprint.portfolio_response(p2_result)

Portfolio Created
Scope: ca_demo_abc
Code: portfolio-ea77
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-09-16 12:46:50.900486+00:00

Portfolio Created
Scope: ca_demo_xyz
Code: portfolio-9904
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-09-16 12:46:51.216058+00:00



Next we will also create a Derived Portfolio with Portfolio A as a parent (which uses CASource1) and assign it CASource2 instead.

_Run the cell below to do this_

In [12]:
# create derived portfolio with second source
derived_request = models.CreateDerivedTransactionPortfolioRequest(
    display_name="derived_portfolio", 
    description=None, 
    code="derived_portfolio-"+guid, 
    parent_portfolio_id=p1_result.id,
    created=effective_date,
    corporate_action_source_id=CASource2, 
    accounting_method=None, 
    sub_holding_keys=None
    )
p_derived = client.derived_transaction_portfolios.create_derived_portfolio(
    scope=scope1, 
    portfolio=derived_request)

prettyprint.portfolio_response(p_derived)

Derived Portfolio Created
Scope: ca_demo_abc
Code: derived_portfolio-b7ba
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-09-16 12:46:51.515593+00:00

   Parent Portfolio Details
   Scope: ca_demo_abc
   Code: portfolio-ea77



_Run the cell below to get the Derived Portfolio details showing a different source to the parent_

In [13]:
details = client.transaction_portfolios.get_details(p_derived.id.scope, p_derived.id.code)
prettyprint.portfolio_details_response(details)

Portfolio Details: 
Detail Origin Portfolio Scope: ca_demo_abc
Detail Origin Portfolio Code: derived_portfolio-b7ba
Base Currency: GBP
Corporate Action Source Id: {'code': 'ca_demo_xyz_b7ba', 'scope': 'ca_demo_xyz'}




#### 3.2 Upserting Transactions
In order to setup our portfolios to receive Corporate Actions, we need to populate them with the instruments affected by the Corporate Actions in the data loaded previously. The cell below contains some helper methods to extract identifiers from the data and to set up the transactions themselves.

In [14]:
LUSID_INSTRUMENT_IDENTIFIER = "Instrument/default/LusidInstrumentId" 
FIGI_INSTRUMENT_IDENTIFIER = "Instrument/default/Figi"
    
def get_identifiers_from_partial_name(part) :
    name = instruments.loc[instruments['instrument_name'].str.contains(part),'instrument_name'].values[0]
    luid = luid_cas.loc[luid_cas['description'].str.contains(part), 'input_instrument_luid'].values[0]
    figi = ca_info.loc[ca_info['description'].str.contains(part), 'input_instrument_figi'].values[0]
    identifiers = {LUSID_INSTRUMENT_IDENTIFIER : luid, FIGI_INSTRUMENT_IDENTIFIER : figi}
    return identifiers

def setup_transaction(guid, name, identifiers, currency, amount) :
    request = models.TransactionRequest(
        transaction_id=name+guid,
        type="StockIn",
        instrument_identifiers=identifiers,
        transaction_date=Mar01,
        settlement_date=Mar05,
        units=100,
        transaction_price=models.TransactionPrice(101),
        total_consideration=models.CurrencyAndAmount(100 * amount, currency),
        source="Client")
    return request

_Run the cell below to get identifiers for each of the intruments_

In [15]:
# Extract Identifiers
apple_identifiers = get_identifiers_from_partial_name("Apple")
trex_identifiers = get_identifiers_from_partial_name("Trex")
express_identifiers = get_identifiers_from_partial_name("Express")
trinity_identifiers = get_identifiers_from_partial_name("Trinity")

_Run the cell below to create transaction requests for each instrument_

In [16]:
guid = GetGuid()

tx_request_apple = setup_transaction(guid, "Apple", apple_identifiers, "USD", 170)
tx_request_trex = setup_transaction(guid, "Trex", trex_identifiers, "USD", 60)
tx_request_express = setup_transaction(guid, "Express", express_identifiers, "USD", 130)
tx_request_trinity = setup_transaction(guid, "Trinity", trinity_identifiers, "USD", 25)

Finally we can upsert the transactions to the portfolios. Since the derived portfolio will inherit the transactions from the parent, we only need to add them to the two first level portfolios.

_Run the cell below to upsert the transactions to the portfolios_

In [17]:
tx = [tx_request_apple, tx_request_express, tx_request_trex, tx_request_trinity]

tx1_response = client.transaction_portfolios.upsert_transactions(
    scope=scope1, 
    code=p1_result.id.code, 
    transactions=tx)
prettyprint.transactions_response(tx1_response, scope1, p1_result.id.code)

tx2_response = client.transaction_portfolios.upsert_transactions(
    scope=scope2, 
    code=p2_result.id.code, 
    transactions=tx)
prettyprint.transactions_response(tx2_response, scope2, p2_result.id.code)

Transactions Successfully Upserted into Portfolio
Scope: ca_demo_abc
Code: portfolio-ea77
Transactions Effective From: 2018-03-01 00:00:00+00:00
Transactions Created On: 2019-09-16 12:46:52.184831+00:00

Transactions Successfully Upserted into Portfolio
Scope: ca_demo_xyz
Code: portfolio-9904
Transactions Effective From: 2018-03-01 00:00:00+00:00
Transactions Created On: 2019-09-16 12:46:52.618704+00:00



#### 3.3 Get Holdings 
We can now get holdings effective at the settlement date of the transactions created above and create some dataframes to easily visualise the information for each portfolio. Note the 3 portfolios should return identical holdings at this point.

_Run the cell below to get holdings and create the dataframes_

In [18]:
# Get Holdings and compare

# Use a property key to get the instrument names of each holding
prop_keys = ['Instrument/default/Name']

# Portfolio 1
holdings_p1 = client.transaction_portfolios.get_holdings(
    scope=scope1, 
    code=p1_result.id.code, 
    effective_at=Mar05,
    property_keys=prop_keys)
# Derived Portfolio 1
holdings_d1 = client.transaction_portfolios.get_holdings(
    scope=scope1, 
    code=p_derived.id.code, 
    effective_at=Mar05,
    property_keys=prop_keys)
# Portfolio 2
holdings_p2 = client.transaction_portfolios.get_holdings(
    scope=scope2, 
    code=p2_result.id.code, 
    effective_at=Mar05,
    property_keys=prop_keys)

headers = ['Portfolio', 'Effective_At', 'Instrument_Uid', 'Instrument_Name', 'Holdings_Type', 'Units', 'Settled_Units', 'Currency', 'Amount']
rows_list = []

# Convert the holidings of each portfolio into dictionary form to more easily create dataframes
def holdings_to_dict(holdings, portfolio_name, rows_list, time):
    print('Number of holdings in portfolio '+portfolio_name+' : ', len(holdings))
    for h in holdings:
        dict1 = {}
        d = {
            'Portfolio' : portfolio_name, 
            'Effective_At' : str(time),
            'Instrument_Uid' : h.instrument_uid, 
            'Instrument_Name' : h.properties['Instrument/default/Name'].value.label_value,
            'Holdings_Type' : h.holding_type, 
            'Units' : h.units, 
            'Settled_Units' :  h.settled_units, 
            'Currency' : h.cost.currency, 
            'Amount' : h.cost.amount
            }
        dict1.update(d) 
        rows_list.append(dict1)
    return rows_list

p1_list = holdings_to_dict(holdings_p1.values, 'portfolio1', [], Mar05)
d1_list = holdings_to_dict(holdings_d1.values, 'derived1', [], Mar05)
p2_list = holdings_to_dict(holdings_p2.values, 'portfolio2', [], Mar05)

# Create a dataframe containing holdings for each portfolio
p1_df = pd.DataFrame(p1_list, columns=headers)  
d1_df = pd.DataFrame(d1_list, columns=headers)
p2_df = pd.DataFrame(p2_list, columns=headers)


Number of holdings in portfolio portfolio1 :  4
Number of holdings in portfolio derived1 :  4
Number of holdings in portfolio portfolio2 :  4


_Run the cell below to see the holdings of Portfolio 1_

In [19]:
p1_df.head()


Portfolio               Effective_At Instrument_Uid  \
0  portfolio1  2018-03-05 00:00:00+00:00  LUID_KVBLERMO   
1  portfolio1  2018-03-05 00:00:00+00:00  LUID_S6ANN8K2   
2  portfolio1  2018-03-05 00:00:00+00:00  LUID_CATB74OJ   
3  portfolio1  2018-03-05 00:00:00+00:00  LUID_GH8D5K6V   

              Instrument_Name Holdings_Type  Units  Settled_Units Currency  \
0           Apple_Nasdaq_AAPL             P  100.0          100.0      USD   
1    ExpressScripts_NYSE_ESRX             P  100.0          100.0      USD   
2              Trex_NYSE_TREX             P  100.0          100.0      USD   
3  TrinityIndustries_NYSE_TRN             P  100.0          100.0      USD   

    Amount  
0  17000.0  
1  13000.0  
2   6000.0  
3   2500.0

_Run the cell below to see the holdings for the Derived Portfolio_

In [20]:
d1_df.head()


Portfolio               Effective_At Instrument_Uid  \
0  derived1  2018-03-05 00:00:00+00:00  LUID_KVBLERMO   
1  derived1  2018-03-05 00:00:00+00:00  LUID_S6ANN8K2   
2  derived1  2018-03-05 00:00:00+00:00  LUID_CATB74OJ   
3  derived1  2018-03-05 00:00:00+00:00  LUID_GH8D5K6V   

              Instrument_Name Holdings_Type  Units  Settled_Units Currency  \
0           Apple_Nasdaq_AAPL             P  100.0          100.0      USD   
1    ExpressScripts_NYSE_ESRX             P  100.0          100.0      USD   
2              Trex_NYSE_TREX             P  100.0          100.0      USD   
3  TrinityIndustries_NYSE_TRN             P  100.0          100.0      USD   

    Amount  
0  17000.0  
1  13000.0  
2   6000.0  
3   2500.0

_Run the cell below to see the holdings of Portfolio 2_

In [21]:
p2_df.head()

Portfolio               Effective_At Instrument_Uid  \
0  portfolio2  2018-03-05 00:00:00+00:00  LUID_KVBLERMO   
1  portfolio2  2018-03-05 00:00:00+00:00  LUID_S6ANN8K2   
2  portfolio2  2018-03-05 00:00:00+00:00  LUID_CATB74OJ   
3  portfolio2  2018-03-05 00:00:00+00:00  LUID_GH8D5K6V   

              Instrument_Name Holdings_Type  Units  Settled_Units Currency  \
0           Apple_Nasdaq_AAPL             P  100.0          100.0      USD   
1    ExpressScripts_NYSE_ESRX             P  100.0          100.0      USD   
2              Trex_NYSE_TREX             P  100.0          100.0      USD   
3  TrinityIndustries_NYSE_TRN             P  100.0          100.0      USD   

    Amount  
0  17000.0  
1  13000.0  
2   6000.0  
3   2500.0

## 4. Corporate Actions
#### 4.1 Setup
A Corporate Action in LUSID is composed of Transitions, which in turn contain one input component and one or more output components. The CSV data loaded earlier contains all the information required to create the requests for each of these parts.

_Run the cell below to set up the transitions for each corporate action_

In [22]:
import math
transitions = {}
actions = []

for index, ca in luid_cas.iterrows():
    # Print the details contained in the CSV for each corporate action
    prettyprint.corporate_action_request_details(ca)
        
    # create transition components
    cat_in = models.CorporateActionTransitionComponentRequest(
        instrument_identifiers={LUSID_INSTRUMENT_IDENTIFIER : ca['input_instrument_luid']}, 
        units_factor=ca['input_units_factor'], 
        cost_factor=ca['input_cost_factor'])
    
    # Determine if the output is Cash or Stock, and create the appropriate transition component
    if (str(ca['output_instrument_luid']) == 'nan'):
        cat_out = models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={"Instrument/default/Currency" : ca['output_instrument_internal'][4:]},
            units_factor=ca['output_units_factor'], 
            cost_factor=ca['output_cost_factor'])
    else :
        cat_out = models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={LUSID_INSTRUMENT_IDENTIFIER : ca['output_instrument_luid']},
            units_factor=ca['output_units_factor'], 
            cost_factor=ca['output_cost_factor'])

    key = ca['code']
    transitions.setdefault(key, [])
    transitions[key].append(ca['action_description'])
    transitions[key].append(cat_in)
    transitions[key].append(cat_out)

CA Code and Type: 5943592342dividend
Announcement Date : 29/01/2019
Ex Date : 02/08/2019
Record Date : 02/11/2019
Payment Date : 14/02/2019
input instrument : LUID_KVBLERMO
Units in : 1 Cost in : 1
output instrument : nan
output internal : CCY_GBP
Units out : 0.017 Cost out : 1.0

CA Code and Type: 5943592343merger
Announcement Date : 03/08/2018
Ex Date : 12/10/2018
Record Date : 13/12/2018
Payment Date : 20/12/2018
input instrument : LUID_S6ANN8K2
Units in : 1 Cost in : 1
output instrument : nan
output internal : CCY_USD
Units out : 48.75 Cost out : 1.0

CA Code and Type: 5943592343merger
Announcement Date : 03/08/2018
Ex Date : 12/10/2018
Record Date : 13/12/2018
Payment Date : 20/12/2018
input instrument : LUID_S6ANN8K2
Units in : 1 Cost in : 1
output instrument : LUID_WCV2P2VV
output internal : nan
Units out : 0.2434 Cost out : 1.9337

CA Code and Type: 5943592347spin-off
Announcement Date : 12/12/2017
Ex Date : 17/10/2018
Record Date : 20/10/2018
Payment Date : 11/01/2018
input in

_Run the cell below to create Corporate Action requests for each type of action_

In [23]:
# Function to convert strings to date objects with timezones
def str_to_tzdate(dtstring):
    date_time_obj = datetime.strptime(dtstring, '%d/%m/%Y')
    timezone = pytz.utc 
    timezone_date_time_obj = timezone.localize(date_time_obj)
    return timezone_date_time_obj

# Iterate through the transitions, turning them into LUSID Corporate Action Requests
for key, values in transitions.items():
    transition_code = key
    transition_type = values[0]
    print(transition_type)
    transition_in = values[1]
    transitions_out = values[2:]
    for x in transitions_out:        
        if isinstance(x, str) :
            transitions_out.remove(x)
    
    temp_transition = models.CorporateActionTransition(
        input_transition=transition_in, 
        output_transitions=transitions_out)
    
    # Extract the data for the corporate action from the LUID corporate actions dataframe
    data = luid_cas[luid_cas['code'] == transition_code]
    
    # Iterate through each row of data and create the Corporate Action Request for the appropriate type
    # of action. 
    for row, d in data.iterrows():
        if transition_type == 'dividend':
            div_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(d['code']),
                announcement_date=Mar08,
                ex_date=Mar09,
                record_date=Mar13, 
                payment_date=Mar15, 
                transitions=[temp_transition])
        if transition_type == 'split' :
            split_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(d['code']),
                announcement_date=str_to_tzdate(d['announcement_date']),
                ex_date=str_to_tzdate(d['ex_date']),
                record_date=str_to_tzdate(d['record_date']), 
                payment_date=str_to_tzdate(d['record_date']), 
                transitions=[temp_transition])
        if transition_type == 'merger' :
            merge_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(d['code']),
                announcement_date=str_to_tzdate(d['announcement_date']),
                ex_date=str_to_tzdate(d['ex_date']),
                record_date=str_to_tzdate(d['record_date']), 
                payment_date=str_to_tzdate(d['record_date']), 
                transitions=[temp_transition])
        if transition_type == 'spin-off' :
            spinoff_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(d['code']),
                announcement_date=str_to_tzdate(d['announcement_date']),
                ex_date=str_to_tzdate(d['ex_date']),
                record_date=str_to_tzdate(d['record_date']), 
                payment_date=str_to_tzdate(d['record_date']), 
                transitions=[temp_transition])

dividend
merger
spin-off
split


#### 4.2. Upsert to different sources
Now that the requests are set up, we will split the Corporate Actions into two lots, one to be Upserted to CASource1 (on portfolio 1), and the second will got to CASource2 (on the derived portfolio).

_Run the cell below to upsert the Corporate Actions_

In [24]:
# Divide the corporate actions into two sets, one for each source
scope_1_actions = [div_ca, split_ca]
scope_2_actions = [merge_ca, spinoff_ca]

# Upsert corporate actions to LUSID
scope1_upsert_response = client.corporate_action_sources.batch_upsert_corporate_actions(
    scope=CASource1.scope, 
    code=CASource1.code, 
    actions=scope_1_actions)

scope2_upsert_response = client.corporate_action_sources.batch_upsert_corporate_actions(
    scope=CASource2.scope, 
    code=CASource2.code, 
    actions=scope_2_actions)

# Print the Upsert responses
prettyprint.batch_upsert_corporate_actions_response(scope1_upsert_response)
prettyprint.batch_upsert_corporate_actions_response(scope2_upsert_response)

Corporate Action Id : 5943592498
Announcement Date : 2018-05-05 00:00:00+00:00
Ex Date : 2018-05-21 00:00:00+00:00
Payment Date : 2018-05-23 00:00:00+00:00
Record Date : 2018-05-23 00:00:00+00:00
   Transitions : 
Input LUID: LUID_CATB74OJ with Cost factor: 1.0  and Unit Factor: 1.0
Output LUID: LUID_CATB74OJ  with Cost Factor: 1.0  and Unit Factor: 2.0

Corporate Action Id : 5943592342
Announcement Date : 2018-03-08 00:00:00+00:00
Ex Date : 2018-03-09 00:00:00+00:00
Payment Date : 2018-03-15 00:00:00+00:00
Record Date : 2018-03-13 00:00:00+00:00
   Transitions : 
Input LUID: LUID_KVBLERMO with Cost factor: 1.0  and Unit Factor: 1.0
Output LUID: CCY_GBP  with Cost Factor: 1.0  and Unit Factor: 0.017

Corporate Action Id : 5943592347
Announcement Date : 2017-12-12 00:00:00+00:00
Ex Date : 2018-10-17 00:00:00+00:00
Payment Date : 2018-10-20 00:00:00+00:00
Record Date : 2018-10-20 00:00:00+00:00
   Transitions : 
Input LUID: LUID_GH8D5K6V with Cost factor: 1.0  and Unit Factor: 3.0
Output

#### 4.3. Get Holdings
Since portfolio 1 and the dervied portfolio have different CA Sources on them, we should only see their respective Corporate Actions applied. To verify this, we will get the holdings for each portfolio once again, this time at the latest time, and add these to the holdings dataframes that were created earlier. 

In [25]:
# Get Holdings and compare
holdings_p1 = client.transaction_portfolios.get_holdings(scope=scope1, code=p1_result.id.code, property_keys=prop_keys)
holdings_d1 = client.transaction_portfolios.get_holdings(scope=scope1, code=p_derived.id.code, property_keys=prop_keys)
holdings_p2 = client.transaction_portfolios.get_holdings(scope=scope2, code=p2_result.id.code, property_keys=prop_keys)

headers = ['Portfolio', 'Effective_At', 'Instrument_Uid', 'Instrument_Name', 'Holdings_Type', 'Units', 'Settled_Units', 'Currency', 'Amount']
rows_list = []

current_p1_list = holdings_to_dict(holdings_p1.values, 'portfolio1', [], datetime.now())
current_d1_list = holdings_to_dict(holdings_d1.values, 'derived1', [], datetime.now())
current_p2_list = holdings_to_dict(holdings_p2.values, 'portfolio2', [], datetime.now())

Number of holdings in portfolio portfolio1 :  5
Number of holdings in portfolio derived1 :  7
Number of holdings in portfolio portfolio2 :  4


For portfolio 1, the dividend results in CCY_GBP being added, and the stock split results in double units of Trex stock. 

_Run the cell below to see the holdings before and after the corporate action for portfolio 1_

In [26]:
p1_df = p1_df.append(current_p1_list, ignore_index=True)
p1_df = p1_df.sort_values(by=['Portfolio', 'Instrument_Uid', 'Effective_At'])
p1_df.head(10)

Portfolio                Effective_At Instrument_Uid  \
8  portfolio1  2019-09-16 13:46:55.347023        CCY_GBP   
2  portfolio1   2018-03-05 00:00:00+00:00  LUID_CATB74OJ   
6  portfolio1  2019-09-16 13:46:55.347023  LUID_CATB74OJ   
3  portfolio1   2018-03-05 00:00:00+00:00  LUID_GH8D5K6V   
7  portfolio1  2019-09-16 13:46:55.347023  LUID_GH8D5K6V   
0  portfolio1   2018-03-05 00:00:00+00:00  LUID_KVBLERMO   
4  portfolio1  2019-09-16 13:46:55.347023  LUID_KVBLERMO   
1  portfolio1   2018-03-05 00:00:00+00:00  LUID_S6ANN8K2   
5  portfolio1  2019-09-16 13:46:55.347023  LUID_S6ANN8K2   

              Instrument_Name Holdings_Type  Units  Settled_Units Currency  \
8                     CCY_GBP             B    1.7            1.7      GBP   
2              Trex_NYSE_TREX             P  100.0          100.0      USD   
6              Trex_NYSE_TREX             P  200.0          200.0      USD   
3  TrinityIndustries_NYSE_TRN             P  100.0          100.0      USD   
7  TrinityIndustries_NYSE_TRN             P  100.0          100.0      USD   
0           Apple_Nasdaq_AAPL             P  100.0          100.0      USD   
4           Apple_Nasdaq_AAPL             P  100.0          100.0      USD   
1    ExpressScripts_NYSE_ESRX             P  100.0          100.0      USD   
5    ExpressScripts_NYSE_ESRX             P  100.0          100.0      USD   

    Amount  
8      1.7  
2   6000.0  
6   6000.0  
3   2500.0  
7   2500.0  
0  17000.0  
4  17000.0  
1  13000.0  
5  13000.0

For the derived portfolio, the merger results in CCY_USD being added and Cigna stock being added, and the spinoff results in Arcosa stock being added. 

_Run the cell below to see the holdings before and after the corporate actions for the derived portfolio_

In [27]:
d1_df = d1_df.append(current_d1_list, ignore_index=True)
d1_df = d1_df.sort_values(by=['Portfolio', 'Instrument_Uid', 'Effective_At'])
d1_df.head(20)

Portfolio                Effective_At Instrument_Uid  \
10  derived1  2019-09-16 13:46:55.347976        CCY_USD   
2   derived1   2018-03-05 00:00:00+00:00  LUID_CATB74OJ   
6   derived1  2019-09-16 13:46:55.347976  LUID_CATB74OJ   
9   derived1  2019-09-16 13:46:55.347976  LUID_E8ZMJAH7   
3   derived1   2018-03-05 00:00:00+00:00  LUID_GH8D5K6V   
7   derived1  2019-09-16 13:46:55.347976  LUID_GH8D5K6V   
0   derived1   2018-03-05 00:00:00+00:00  LUID_KVBLERMO   
4   derived1  2019-09-16 13:46:55.347976  LUID_KVBLERMO   
1   derived1   2018-03-05 00:00:00+00:00  LUID_S6ANN8K2   
5   derived1  2019-09-16 13:46:55.347976  LUID_S6ANN8K2   
8   derived1  2019-09-16 13:46:55.347976  LUID_WCV2P2VV   

               Instrument_Name Holdings_Type   Units  Settled_Units Currency  \
10                     CCY_USD             B  4875.0         4875.0      USD   
2               Trex_NYSE_TREX             P   100.0          100.0      USD   
6               Trex_NYSE_TREX             P   100.0          100.0      USD   
9              Arcosa_NYSE_ACA             P    33.0           33.0      USD   
3   TrinityIndustries_NYSE_TRN             P   100.0          100.0      USD   
7   TrinityIndustries_NYSE_TRN             P   100.0          100.0      USD   
0            Apple_Nasdaq_AAPL             P   100.0          100.0      USD   
4            Apple_Nasdaq_AAPL             P   100.0          100.0      USD   
1     ExpressScripts_NYSE_ESRX             P   100.0          100.0      USD   
5     ExpressScripts_NYSE_ESRX             P   100.0          100.0      USD   
8                Cigna_NYSE_CI             P    24.0           24.0      USD   

     Amount  
10   4875.0  
2    6000.0  
6    6000.0  
9    2581.0  
3    2500.0  
7    2500.0  
0   17000.0  
4   17000.0  
1   13000.0  
5   13000.0  
8   25138.1

Since portfolio 2 does not have a corporate action source assigned, there should be no change in the holdings. Notice all the amounts are the same, and there are no new Instrument Uids added to the list.

_Run the cell below to see the holdings before and after the corporate actions for portfolio 2_

In [28]:
p2_df = p2_df.append(current_p2_list, ignore_index=True)
p2_df = p2_df.sort_values(by=['Portfolio', 'Instrument_Uid', 'Effective_At'])
p2_df.head(10)

Portfolio                Effective_At Instrument_Uid  \
2  portfolio2   2018-03-05 00:00:00+00:00  LUID_CATB74OJ   
6  portfolio2  2019-09-16 13:46:55.347976  LUID_CATB74OJ   
3  portfolio2   2018-03-05 00:00:00+00:00  LUID_GH8D5K6V   
7  portfolio2  2019-09-16 13:46:55.347976  LUID_GH8D5K6V   
0  portfolio2   2018-03-05 00:00:00+00:00  LUID_KVBLERMO   
4  portfolio2  2019-09-16 13:46:55.347976  LUID_KVBLERMO   
1  portfolio2   2018-03-05 00:00:00+00:00  LUID_S6ANN8K2   
5  portfolio2  2019-09-16 13:46:55.347976  LUID_S6ANN8K2   

              Instrument_Name Holdings_Type  Units  Settled_Units Currency  \
2              Trex_NYSE_TREX             P  100.0          100.0      USD   
6              Trex_NYSE_TREX             P  100.0          100.0      USD   
3  TrinityIndustries_NYSE_TRN             P  100.0          100.0      USD   
7  TrinityIndustries_NYSE_TRN             P  100.0          100.0      USD   
0           Apple_Nasdaq_AAPL             P  100.0          100.0      USD   
4           Apple_Nasdaq_AAPL             P  100.0          100.0      USD   
1    ExpressScripts_NYSE_ESRX             P  100.0          100.0      USD   
5    ExpressScripts_NYSE_ESRX             P  100.0          100.0      USD   

    Amount  
2   6000.0  
6   6000.0  
3   2500.0  
7   2500.0  
0  17000.0  
4  17000.0  
1  13000.0  
5  13000.0

We can also query a source for all the corporate actions within it.

_Run the cell below to obtain the corporate actions in each source and print them out_

In [29]:
get_result_1 = client.corporate_action_sources.get_corporate_actions(scope1, code1)
prettyprint.get_corporate_actions_response(scope1, code1, get_result_1)

get_result_2 = client.corporate_action_sources.get_corporate_actions(scope2, code2)
prettyprint.get_corporate_actions_response(scope2, code2, get_result_2)


Corporate Action Id : 5943592342
Source Id : 
    Scope : ca_demo_abc
    Code : ca_demo_abc_b7ba
Announcement Date : 2018-03-08 00:00:00+00:00
Ex Date : 2018-03-09 00:00:00+00:00
Payment Date : 2018-03-15 00:00:00+00:00
Record Date : 2018-03-13 00:00:00+00:00
   Transitions : 
Input LUID: LUID_KVBLERMO with Cost factor: 1.0  and Unit Factor: 1.0
Output LUID: CCY_GBP  with Cost Factor: 1.0  and Unit Factor: 0.017

Corporate Action Id : 5943592498
Source Id : 
    Scope : ca_demo_abc
    Code : ca_demo_abc_b7ba
Announcement Date : 2018-05-05 00:00:00+00:00
Ex Date : 2018-05-21 00:00:00+00:00
Payment Date : 2018-05-23 00:00:00+00:00
Record Date : 2018-05-23 00:00:00+00:00
   Transitions : 
Input LUID: LUID_CATB74OJ with Cost factor: 1.0  and Unit Factor: 1.0
Output LUID: LUID_CATB74OJ  with Cost Factor: 1.0  and Unit Factor: 2.0

Corporate Action Id : 5943592343
Source Id : 
    Scope : ca_demo_xyz
    Code : ca_demo_xyz_b7ba
Announcement Date : 2018-08-03 00:00:00+00:00
Ex Date : 2018-1

It is also possible to delete a Corporate Action Source, though there shouldn't be a need for this. Deleting a source will not remove it from any Portfolio Details however, so those will need updating manually to a null or new source. Portfolio Detail updates are demonstrated in the Portfolios notebook. Corporate Actions themselves cannot be deleted.

_Run the cell below to delete the two corporate action sources that were created at the start of this notebook_

In [30]:
delete1 = client.corporate_action_sources.delete_corporate_action_source(scope1, code1)
delete2 = client.corporate_action_sources.delete_corporate_action_source(scope2, code2)

print("Source1 deleted at : " + str(delete1.as_at))
print("Source2 deleted at : " + str(delete2.as_at))

Source1 deleted at : 2019-09-16 12:46:55.659621+00:00
Source2 deleted at : 2019-09-16 12:46:55.887157+00:00
